# Sequential

###  sequential 有三种方式，常用的有两种

In [1]:
from torch import nn
from torchsummary import summary

In [2]:
net1 = nn.Sequential()
net1.add_module('conv', nn.Conv2d(3, 3, 3))
net1.add_module('batchnorm', nn.BatchNorm2d(3))
net1.add_module('activation_layer', nn.ReLU())

print('net1:', net1)

net1: Sequential(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation_layer): ReLU()
)


In [3]:
net2 = nn.Sequential(
        nn.Conv2d(3, 3, 3),
        nn.BatchNorm2d(3),
        nn.ReLU()
        )

print('net2:', net2)

net2: Sequential(
  (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)


可根据名字或序号取出子module

In [4]:
net1.conv, net2[0]

(Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)),
 Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)))

# ModuleList

ModuleList是Module的子类，当在Module中使用它的时候，就能自动识别为子module。

当添加 nn.ModuleList 作为 nn.Module 对象的一个成员时（即当我们添加模块到我们的网络时），所有 nn.ModuleList 内部的 nn.Module 的 parameter 也被添加作为 我们的网络的 parameter。

In [5]:
class MyModule_1(nn.Module):
    def __init__(self):
        super(MyModule_1, self).__init__()
        # self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(5)])
        self.linears = nn.ModuleList([nn.Conv2d(3, 3, 3), nn.BatchNorm2d(3), nn.ReLU()])
        
    def forward(self, x):
        # ModuleList can be indexed using ints
        for i in range(len(self.linears)):
            x = self.linears[i](x)
        return x

class MyModule_2(nn.Module):
    def __init__(self):
        super(MyModule_2, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(5)])
 
    def forward(self, x):
        # ModuleList can act as an iterable
        for i, l in enumerate(self.linears):
            x = l(x)
        return x

In [6]:
net3 = MyModule_1()
net4 = MyModule_2()
print(net3)
print(net4)
summary(net3, (3,10,10))
summary(net4, (10,))

MyModule_1(
  (linears): ModuleList(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)
MyModule_2(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 3, 8, 8]              84
       BatchNorm2d-2              [-1, 3, 8, 8]               6
              ReLU-3              [-1, 3, 8, 8]               0
Total params: 90
Trainable params: 90
Non-trainable params: 0
----------------------------------------------------------------
Input

# Sequential 和 ModuleList结合

In [7]:
class test_net(nn.Module):
    def __init__(self, num_layers, input_size):
        super(test_net, self).__init__()
        self.num_layers= num_layers
        self.linear_1 = nn.Linear(input_size, 5)
        self.middle = nn.ModuleList([nn.Linear(5,5) for x in range(num_layers)])
        self.output = nn.Linear(5,2)
    
    def forward(self, x):
        fwd = nn.Sequential(self.linear_1, *self.middle, self.output)
        return fwd(x)

net5 = test_net(5, 10)
print(net5)
summary(net5,(10,))

test_net(
  (linear_1): Linear(in_features=10, out_features=5, bias=True)
  (middle): ModuleList(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=5, bias=True)
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): Linear(in_features=5, out_features=5, bias=True)
    (4): Linear(in_features=5, out_features=5, bias=True)
  )
  (output): Linear(in_features=5, out_features=2, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 5]              55
            Linear-2                    [-1, 5]              30
            Linear-3                    [-1, 5]              30
            Linear-4                    [-1, 5]              30
            Linear-5                    [-1, 5]              30
            Linear-6                    [-1, 5]              30
            Linear-7            

# ModuleList 的extend 和 append方法

nn.moduleList定义对象后，有extend和append方法，用法和python中一样，extend是添加另一个modulelist  append是添加另一个module

In [13]:
class LinearNet(nn.Module):
    def __init__(self, input_size, num_layers, layers_size, output_size):
        super(LinearNet, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(input_size, layers_size)])
        self.linears.extend([nn.Linear(layers_size, layers_size) for i in range(1, self.num_layers-1)])
        self.linears.append(nn.Linear(layers_size, output_size))

# ModuleList 与普通List

普通list中的子module并不能被主module所识别，而ModuleList中的子module能够被主module所识别。这意味着如果用list保存子module，将无法调整其参数，因其未加入到主module的参数中。

In [20]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.list = [nn.Linear(3, 4), nn.ReLU()]
        self.module_list = nn.ModuleList([nn.Conv2d(3, 3, 3), nn.ReLU()])
    def forward(self):
        pass
        
model = MyModule()
print(model)

MyModule(
  (module_list): ModuleList(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
)
